In [1]:
from premise import *
import brightway2 as bw
bw.projects.set_current("ei39")

In [2]:
combinations = [
    {"range time":False, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 5, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":True, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":True, "lead time":False, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 1, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 2, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 3, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":True, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":True, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":True, "lead time":False, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 1, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 2, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 3, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":True, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 4, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":True, "lead time":False, "capital replacement rate":False, "measurement": 4, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":True, "measurement": 4, "weighted slope start": 0.75, "weighted slope end": 1.00}
]

In [3]:
scenarios = [
    #{"model": "remind", "pathway":"SSP2-PkBudg500", "year": 2030},
    #{"model": "remind", "pathway":"SSP2-PkBudg500", "year": 2050},
    #{"model": "remind", "pathway":"SSP2-PkBudg500", "year": 2070},
    #{"model": "remind", "pathway":"SSP2-PkBudg500", "year": 2030},
    #{"model": "remind", "pathway":"SSP2-PkBudg500", "year": 2050},
    #{"model": "remind", "pathway":"SSP2-PkBudg500", "year": 2010},
    {"model": "remind", "pathway":"SSP1-Base", "year": 2005},
    {"model": "image", "pathway":"SSP2-Base", "year": 2100},
    {"model": "remind", "pathway":"SSP5-Base", "year": 2050},
    
]

In [4]:
ndb = NewDatabase(
        scenarios = scenarios,
        source_db= "ecoinvent 3.9 cutoff",
        source_version="3.9",
        #source_type=source,
        #source_file_path=f"/Users/romain/Documents/ecoinvent {ei}_cutoff_ecoSpold02/datasets" if source == "ecospold" else None, # <-- this is NEW
        #source_version=ei,
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        system_model="cutoff",
        system_args=combinations[0]
)

premise v.(1, 5, 0, 'alpha')
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+----

In [5]:
ndb.update_all()

`update_all()` will skip the following steps:
update_two_wheelers(), update_cars(), and update_buses()
If you want to update these steps, please run them separately afterwards.

////////////////// MEDIUM AND HEAVY DUTY TRUCKS ////////////////////
BR-South-eastern/Mid-western grid
CN-NECG
Extracted 1 worksheets in 3.63 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Create fleet average vehicles...
Vehicle fleet data is not available before 2015. Hence, 2015 is used as fleet year.
Done!
Extracted 1 worksheets in 3.94 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Create fleet average vehicles...
Vehicle fleet data is not available beyond 2050. Hence, 2050 is used as fleet year.
Done!
Extracted 1 worksheets in 3.66 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Create fleet average vehicles...
Done!

/////////////////////////// ELECTRICITY ////////////////////////////
Update efficiency of

In [6]:
ndb.write_db_to_brightway()

Write new database(s) to Brightway2.
Prepare database 1.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!


KeyError: "Can't find linking activity for exchange:\n{'amount': 0.000227,\n 'location': 'RoW',\n 'name': 'treatment of wastewater, from residence, capacity 1.1E10l/year',\n 'product': 'wastewater, from residence',\n 'type': 'technosphere',\n 'unit': 'cubic meter'}"

In [16]:
ndb.scenarios[1]["iam data"].fuel_markets.sel(region="NEU").variables.values

array(['natural gas', 'biomethane', 'diesel', 'gasoline',
       'petrol, synthetic, from electrolysis',
       'petrol, synthetic, from coal',
       'petrol, synthetic, from coal, with CCS',
       'diesel, synthetic, from electrolysis',
       'diesel, synthetic, from coal',
       'diesel, synthetic, from coal, with CCS',
       'diesel, synthetic, from wood',
       'diesel, synthetic, from wood, with CCS', 'hydrogen, electrolysis',
       'hydrogen, biomass', 'hydrogen, biomass, with CCS',
       'hydrogen, coal', 'hydrogen, nat. gas',
       'hydrogen, nat. gas, with CCS', 'biodiesel, oil',
       'bioethanol, wood', 'bioethanol, wood, with CCS',
       'bioethanol, grass', 'bioethanol, sugar'], dtype='<U38')

In [8]:
import numpy as np
for ds in ndb.scenarios[1]["database"]:
    if ds["name"] == ("carbon dioxide, captured from atmosphere, with a solvent-based direct air capture system, 1MtCO2, with waste heat, and grid electricity",
                            "carbon dioxide, captured from atmosphere"):
            print(ds["name"], ds["location"])

In [9]:
for ds in ndb.scenarios[1]["database"]:
    if "carbon dioxide, captured" in ds["name"]:
        print(ds["name"], ds["location"])

carbon dioxide, captured from atmosphere, with a solvent-based direct air capture system, 1MtCO2 RER
carbon dioxide, captured from atmosphere and stored, with a solvent-based direct air capture system, 1MtCO2 RER
carbon dioxide, captured from atmosphere, with a sorbent-based direct air capture system, 100ktCO2 RER
carbon dioxide, captured from atmosphere and stored, with a sorbent-based direct air capture system, 100ktCO2 RER
carbon dioxide, captured at cement production plant, with underground storage, post, 200 km RER
carbon dioxide, captured at cement production plant, for subsequent reuse RER
carbon dioxide, captured at cement production plant, integrated in fuel production chain RER


In [6]:
ndb.scenarios[0]["iam data"].fuel_markets.sel(region="WEU", year=2050)

<xarray.DataArray 'value' (variables: 20)>
array([0.33881825, 0.33881825, 0.00972326, 0.00897532, 0.01421092,
       0.01196709, 0.        , 0.        , 0.01570681, 0.01570681,
       0.02318624, 0.02019447, 0.00224383, 0.        , 0.01944652,
       0.01944652, 0.03515333, 0.04861631, 0.03515333, 0.04263276])
Coordinates:
    region     <U3 'WEU'
  * variables  (variables) <U39 'diesel' ... 'methanol, grass, with CCS'
    year       int64 2050

In [8]:
ndb.scenarios[0]["iam data"].fuel_markets.sel(region="WEU")

<xarray.DataArray 'value' (variables: 20, year: 1)>
array([[ 0.37624585],
       [ 0.37624585],
       [ 0.01079734],
       [-0.        ],
       [ 0.01578073],
       [-0.        ],
       [-0.        ],
       [-0.        ],
       [ 0.01744186],
       [-0.        ],
       [ 0.02574751],
       [-0.        ],
       [ 0.00249169],
       [-0.        ],
       [ 0.02159468],
       [ 0.02159468],
       [ 0.03903654],
       [ 0.05398671],
       [ 0.03903654],
       [-0.        ]])
Coordinates:
    region     <U3 'WEU'
  * variables  (variables) <U39 'diesel' ... 'methanol, grass, with CCS'
  * year       (year) int64 2050

In [5]:
ndb.update_electricity()


/////////////////////////// ELECTRICITY ////////////////////////////
Update natural gas extraction datasets.
Update efficiency of solar PV.
Create biomass markets.
Replace biomass inputs.
Create region-specific power plants.
Empty old electricity datasets
Create high voltage markets.
{'Biomass CHP': 0.0, 'Biomass CHP CCS': 0.0, 'Biomass ST': 0.0, 'Biomass IGCC CCS': 0.0, 'Biomass IGCC': 0.0, 'Coal PC': 0.0, 'Coal IGCC': 0.0, 'Coal IGCC CCS': 0.0, 'Coal CHP': 0.0, 'Coal CHP CCS': 0.0, 'Gas OC': 0.002617801047120419, 'Gas CC': 0.002617801047120419, 'Gas CHP': 0.0, 'Gas CHP CCS': 0.0, 'Gas CC CCS': 0.013089005235602096, 'Geothermal': 0.0, 'Hydro': 0.7251308900523561, 'Nuclear': 0.0, 'Oil ST': 0.0, 'Oil CC': 0.0, 'Oil CC CCS': 0.002617801047120419, 'Oil CHP': 0.0, 'Oil CHP CCS': 0.0, 'Solar CSP': 0.005235602094240838, 'Solar PV Centralized': 0.052356020942408384, 'Solar PV Residential': 0.04973821989528796, 'Wind Onshore': 0.12827225130890052, 'Wind Offshore': 0.018324607329842934}
{'Biom

In [6]:
ndb.scenarios[0]["iam data"].fuel_markets

<xarray.DataArray 'value' (region: 27, variables: 20)>
array([[4.42067736e-01, 4.42067736e-01, 1.78253119e-03, 3.56506239e-03,
        3.56506239e-03, 5.34759358e-03, 0.00000000e+00, 0.00000000e+00,
        3.56506239e-03, 7.13012478e-03, 7.13012478e-03, 8.91265597e-03,
        0.00000000e+00, 0.00000000e+00, 8.91265597e-03, 1.06951872e-02,
        8.91265597e-03, 1.42602496e-02, 1.42602496e-02, 1.78253119e-02],
       [4.46096654e-01, 4.46096654e-01, 3.71747212e-03, 3.71747212e-03,
        3.71747212e-03, 3.71747212e-03, 0.00000000e+00, 0.00000000e+00,
        3.71747212e-03, 7.43494424e-03, 7.43494424e-03, 7.43494424e-03,
        0.00000000e+00, 0.00000000e+00, 3.71747212e-03, 3.71747212e-03,
        1.11524164e-02, 1.48698885e-02, 1.48698885e-02, 1.85873606e-02],
       [3.73076923e-01, 3.73076923e-01, 7.69230769e-03, 7.69230769e-03,
        1.53846154e-02, 7.69230769e-03, 0.00000000e+00, 0.00000000e+00,
        1.15384615e-02, 1.15384615e-02, 2.30769231e-02, 1.53846154e-02,
        0.00000000e+00, 0.00000000e+00, 1.53846154e-02, 7.69230769e-03,
        3.07692308e-02, 4.61538462e-02, 2.30769231e-02, 3.07692308e-02],
       [4.37277945e-01, 4.37277945e-01, 3.82618202e-03, 3.00628587e-03,
        6.55916917e-03, 4.91937688e-03, 0.00000000e+00, 0.00000000e+00,
        6.01257174e-03, 4.64607816e-03, 1.03853512e-02, 7.92566275e-03,
        1.36649358e-03, 0.00000000e+00, 8.47226018e-03, 6.01257174e-03,
        1.33916371e-02, 2.10440011e-02, 1.09319486e-02, 1.69445204e-02],
...
       [4.68577729e-01, 4.68577729e-01, 1.92943771e-03, 1.37816979e-03,
        3.03197354e-03, 2.20507166e-03, 0.00000000e+00, 0.00000000e+00,
        3.30760750e-03, 2.48070562e-03, 4.96141125e-03, 3.58324146e-03,
        5.51267916e-04, 0.00000000e+00, 4.41014333e-03, 3.30760750e-03,
        7.44211687e-03, 1.04740904e-02, 5.78831312e-03, 7.99338479e-03],
       [4.33889602e-01, 4.33889602e-01, 3.85109114e-03, 3.85109114e-03,
        6.41848524e-03, 3.85109114e-03, 0.00000000e+00, 0.00000000e+00,
        7.70218228e-03, 6.41848524e-03, 1.02695764e-02, 7.70218228e-03,
        1.28369705e-03, 0.00000000e+00, 6.41848524e-03, 6.41848524e-03,
        1.66880616e-02, 2.18228498e-02, 1.41206675e-02, 1.54043646e-02],
       [3.38818250e-01, 3.38818250e-01, 9.72326103e-03, 8.97531788e-03,
        1.42109200e-02, 1.19670905e-02, 0.00000000e+00, 0.00000000e+00,
        1.57068063e-02, 1.57068063e-02, 2.31862378e-02, 2.01944652e-02,
        2.24382947e-03, 0.00000000e+00, 1.94465221e-02, 1.94465221e-02,
        3.51533283e-02, 4.86163052e-02, 3.51533283e-02, 4.26327599e-02],
       [4.48122840e-01, 4.48122840e-01, 3.10241910e-03, 2.35626767e-03,
        5.30160226e-03, 3.69148602e-03, 0.00000000e+00, 0.00000000e+00,
        5.22306001e-03, 4.00565504e-03, 8.48256362e-03, 6.16556708e-03,
        1.06032045e-03, 0.00000000e+00, 7.06880302e-03, 5.14451775e-03,
        1.18598806e-02, 1.75934653e-02, 9.38579956e-03, 1.33129123e-02]])
Coordinates:
  * region     (region) object 'BRA' 'CAN' 'CEU' 'CHN' ... 'WAF' 'WEU' 'World'
  * variables  (variables) <U39 'diesel' ... 'methanol, grass, with CCS'
    year       int64 2050

In [6]:
ndb.generate_scenario_report()

Generate scenario report.
Report saved under /Users/romain/GitHub/premise/premise/data/export/scenario_report.


In [7]:
ndb.generate_change_report()

Generate change report.
Report saved under /Users/romain/GitHub/premise/dev.


In [4]:
ndb.update_all()

`update_all()` will skip the following steps:
update_two_wheelers(), update_cars(), and update_buses()
If you want to update these steps, please run them separately afterwards.

////////////////// MEDIUM AND HEAVY DUTY TRUCKS ////////////////////
Extracted 1 worksheets in 3.83 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Create fleet average vehicles...
Vehicle fleet data is not available before 2015. Hence, 2015 is used as fleet year.
Done!
Extracted 1 worksheets in 3.57 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Create fleet average vehicles...
Done!

/////////////////////////// ELECTRICITY ////////////////////////////
Update natural gas extraction datasets.
Update efficiency of solar PV.
Create biomass markets.
Replace biomass inputs.
Create region-specific power plants.
Empty old electricity datasets
Create high voltage markets.
Create medium voltage markets.
Create low voltage markets.
Done!
Adjust efficienc

In [5]:
ndb.generate_scenario_report()

Generate scenario report.
Report saved under /Users/romain/GitHub/premise/premise/data/export/scenario_report.


In [6]:
ndb.generate_change_report()

Generate change report.
Report saved under /Users/romain/GitHub/premise/dev.


In [5]:
ndb.scenarios[0]["iam data"].gains_data_IAM

<xarray.DataArray 'value' (region: 12, sector: 56, year: 13, pollutant: 12)>
array([[[[           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         ...,
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan]],

        [[3.14199500e+01, 1.05683770e+03, 2.84628230e+00, ...,
          6.45356613e+00, 3.91388275e+01,            nan],
         [3.13566967e+01, 1.20766658e+03, 2.86307711e+00, ...,
          6.47730792e+00, 4.04007525e+01,            nan],
         [2.78248398e+01, 1.12465874e+03, 2.86826380e+00, ...,
          6.48464003e+00, 3.51287927e+01,            nan],
...
         [           nan,            nan,            nan, ...,
                     nan,            nan, 1.14077533e+00],
         [           nan,            nan,            nan, ...,
                     nan,            nan, 1.14334724e+00],
         [           nan,            nan,            nan, ...,
                     nan,            nan, 1.14507459e+00]],

        [[           nan,            nan,            nan, ...,
                     nan,            nan, 1.96670885e+01],
         [           nan,            nan,            nan, ...,
                     nan,            nan, 1.75838425e+01],
         [           nan,            nan,            nan, ...,
                     nan,            nan, 1.62351222e+01],
         ...,
         [           nan,            nan,            nan, ...,
                     nan,            nan, 1.04798465e+01],
         [           nan,            nan,            nan, ...,
                     nan,            nan, 1.03397167e+01],
         [           nan,            nan,            nan, ...,
                     nan,            nan, 1.02066835e+01]]]])
Coordinates:
  * region     (region) <U3 'SSA' 'MEA' 'OAS' 'LAM' ... 'JPN' 'REF' 'USA' 'NEU'
  * sector     (sector) object 'AACID' 'CEMENT' ... 'Waste_Water_Municipal'
  * year       (year) int64 1990 1995 2000 2005 2010 ... 2035 2040 2045 2050
  * pollutant  (pollutant) object 'PM25' 'PM_OC' 'SO2' ... 'NOx' 'PM10' 'CH4'

In [4]:
ndb.update_all()

`update_all()` will skip the following steps:
update_two_wheelers(), update_cars(), and update_buses()
If you want to update these steps, please run them separately afterwards.

////////////////// MEDIUM AND HEAVY DUTY TRUCKS ////////////////////
Extracted 1 worksheets in 3.82 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Create fleet average vehicles...
Vehicle fleet data is not available before 2015. Hence, 2015 is used as fleet year.
Done!
Extracted 1 worksheets in 3.88 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Create fleet average vehicles...
Done!

/////////////////////////// ELECTRICITY ////////////////////////////
Update natural gas extraction datasets.
Update efficiency of solar PV.
Create biomass markets.
Replace biomass inputs.
Create region-specific power plants.
Empty old electricity datasets
Create high voltage markets.
Create medium voltage markets.
Create low voltage markets.
Done!
Adjust efficienc

In [5]:
ndb.write_db_to_brightway(["test_dac", "test_dac2"])

Write new database(s) to Brightway2.
Prepare database 1.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!
Database test_dac already exists: it will be overwritten.
Vacuuming database 


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:23


Title: Writing activities to SQLite3 database:
  Started: 03/12/2023 19:23:55
  Finished: 03/12/2023 19:24:18
  Total time elapsed: 00:00:23
  CPU %: 97.70
  Memory %: 12.21
Created database: test_dac
Prepare database 2.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!
Database test_dac2 already exists: it will be overwritten.
Vacuuming database 


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:26


Title: Writing activities to SQLite3 database:
  Started: 03/12/2023 19:30:28
  Finished: 03/12/2023 19:30:55
  Total time elapsed: 00:00:26
  CPU %: 97.80
  Memory %: 13.57
Created database: test_dac2
Generate change report.
Report saved under /Users/romain/GitHub/premise/dev.


In [14]:
from premise.geomap import Geomap

In [15]:
Geomap()

TypeError: __init__() missing 1 required positional argument: 'model'

In [ ]:
Geomap(model="remind").geo.keys()

In [7]:
ndb.scenarios[0]["iam data"].carbon_capture_rate.sel(variables="steel")

<xarray.DataArray 'value' (region: 13, year: 19)>
array([[0.        , 0.        , 0.        , 0.        , 0.33019003,
        0.53139234, 0.62505028, 0.67227688, 0.84901398, 0.95673481,
        0.76112484, 0.39062176, 0.37948844, 0.48834529, 0.46170762,
        0.44539283, 0.46818615, 0.47528708, 0.48070597],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.33532806, 0.74346186, 0.8676833 , 0.90584655, 0.86923267,
        0.8462794 , 0.79730203, 0.53459195, 0.59468405, 0.67147208,
        0.60153835, 0.58546995, 0.60250309, 0.62137637],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.54307192, 0.55639549, 0.67163847, 0.74367888, 0.72687844,
        0.61020549, 0.66493675, 0.67037705, 0.64960472, 0.60630547,
        0.55740007, 0.59257289, 0.68211204, 0.73477465],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.10576877, 0.41524998, 1.        , 1.        ,
        1.        , 1.        , 1.        , 0.79593828, 0.64230665,
        0.47422315, 0.38261199, 0.48202266, 0.53420044],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.58325467, 0.79404945, 0.95381854, 0.89438977, 0.8822955 ,
        0.73698279, 0.61350311, 0.47755719, 0.48849111, 0.48252445,
        0.47602982, 0.54736823, 0.78228041, 1.        ],
...
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.27810245, 0.46287488, 0.49662658, 0.5812504 , 0.72269285,
        0.72169706, 0.70541097, 0.61527175, 0.60585798, 0.57362782,
        0.46068575, 0.45377411, 0.43800024, 0.48022694],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.38086168, 0.52867233, 0.57483135, 0.70624219, 0.79547637,
        0.84541732, 0.72561481, 0.69867288, 0.76644286, 0.71722847,
        0.65863303, 0.60858112, 0.66180708, 0.70857874],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.21520316, 0.60847721, 0.67627761, 0.6004029 ,
        0.70287344, 0.74105013, 0.67737603, 0.56020373, 0.4482957 ,
        0.34676261, 0.33642177, 0.33643235, 0.3426008 ],
       [0.        , 0.        , 0.        , 0.        , 0.24079193,
        0.4092863 , 0.4571652 , 0.53625417, 0.59208843, 0.65332675,
        0.75073342, 0.73546282, 0.72250162, 0.79916222, 0.74137873,
        0.73951367, 0.9348905 , 0.90396892, 0.91128312],
       [0.        , 0.        , 0.        , 0.        , 0.00492589,
        0.24284627, 0.35857095, 0.54212608, 0.78737049, 0.81366381,
        0.8319114 , 0.77653552, 0.6531212 , 0.60368055, 0.56143772,
        0.5155564 , 0.49497821, 0.52331794, 0.56158485]])
Coordinates:
  * region     (region) object 'CAZ' 'CHA' 'EUR' 'IND' ... 'SSA' 'USA' 'World'
  * year       (year) int64 2005 2010 2015 2020 2025 ... 2100 2110 2130 2150
    variables  <U6 'steel'

In [8]:
for ds in ndb.scenarios[0]["database"]:
    for e in ds["exchanges"]:
        if e["name"].startswith("CO2 capture, at cement production plant"):
            print(e["name"], e.get("location"), e["amount"])
            print(ds["name"])
            print("--------------------")

CO2 capture, at cement production plant, with underground storage, post, 200 km CAZ 0.6530387843654598
clinker production
--------------------
CO2 capture, at cement production plant, with underground storage, post, 200 km CHA 0.7453367066854295
clinker production
--------------------
CO2 capture, at cement production plant, with underground storage, post, 200 km EUR 0.7076741025455454
clinker production
--------------------
CO2 capture, at cement production plant, with underground storage, post, 200 km IND 0.6675704513410325
clinker production
--------------------
CO2 capture, at cement production plant, with underground storage, post, 200 km JPN 0.733181863891424
clinker production
--------------------
CO2 capture, at cement production plant, with underground storage, post, 200 km LAM 0.7281448684481158
clinker production
--------------------
CO2 capture, at cement production plant, with underground storage, post, 200 km MEA 0.6067184050031128
clinker production
--------------------
